In [1]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as tf
import torch.optim as optim
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt


In [2]:
device  = torch.device("cuda: 0 " if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [3]:
def load_img(image):
  img = Image.open(image).convert('RGB')
  img = img.resize((600,400))
  im_transform = tf.Compose(
      [
       tf.ToTensor(),
       tf.Normalize((0.485,0.456,0.406),(0.229,0.224,0.225))
      ]
  )
  img = im_transform(img)
 
  img = torch.unsqueeze(img,0)
  return img

In [4]:
def im_convert(tensor,index):
 
  image = tensor.to("cpu").clone().detach()
  image = nn.Upsample(scale_factor=2.0,mode='bilinear')(image)
  image = image.numpy().squeeze()
  image = image.transpose(1,2,0)
  image = image * np.array((0.229,0.224,0.225)) + np.array((0.485, 0.456,0.406))
  image = image.clip(0,1)
  image = (image*np.array([255,255,255])).astype(np.uint8)
  image = Image.fromarray(image)
  image.save('seq'+str(index)+'.jpg')
  
  return image

In [5]:
def saveIamge(tensor):
  image = tensor.cpu().clone().detach()
  image = torch.squeeze(image).transpose(1,2,0).numpy()
  image = image*np.array((0.229,0.224,0.225))+np.array((0.485,0.456,0.406))
  image = image.clip(0,1)
  image = tf.ToPILImage()(image)
  image.save('/content/art.jpg')

In [6]:
content_layer_index = 22
style_layer_indexes = [ 0 ,5, 10, 19, 25, 34 ]
style_layer_indexes.append(content_layer_index)
indexes = sorted(style_layer_indexes)
print(indexes)

[0, 5, 10, 19, 22, 25, 34]


In [7]:
def gram_matrix(tensor):
    batch,channel,width,height = tensor.shape
    tensor = torch.squeeze(tensor)
    tensor = tensor.view(channel,height*width)
    return torch.mm(tensor,tensor.t())

In [8]:
vgg_19 = models.vgg19(pretrained=True)
for params in vgg_19.parameters():
  params.requires_grad = False
vgg_19.cuda().eval()
for i, layer in enumerate(vgg_19.features):
  if isinstance(layer, torch.nn.MaxPool2d):
    vgg_19.features[i] = torch.nn.AvgPool2d(kernel_size=2, stride=2, padding=0)
vgg_19

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (17): ReLU(inplace=True)
    (18): AvgPool

In [9]:
def get_feature(image,model,indexes):
  feature = list()
  x = image
  for i , layer in enumerate(model.features):
    x = layer(x)
    if i in indexes:
      
      feature.append(x)
  return feature

In [10]:
contentImage = load_img('content.jpg')
styleImage = load_img('style.jpg')
contentImage , styleImage = contentImage.cuda() , styleImage.cuda()
contentFeatures = get_feature(contentImage,vgg_19,indexes)[4]
style_features = get_feature(styleImage,vgg_19,indexes)
del style_features[4]

In [11]:
len(style_features)

6

In [12]:
style_gram = [gram_matrix(style_features[i]) for i in range(len(style_features))]

In [13]:
len(style_gram)
style_gram[0].requires_grad

False

In [14]:
style_gram_weights = [0.75,0.5,0.5,0.3,0.3,0.3]
len(style_gram_weights)

6

In [15]:
target = torch.randn_like(contentImage).requires_grad_(True).cuda()

In [16]:
optimizer  = optim.Adam([target],lr = 0.09)


In [17]:
def train(epoch,contentWeight,styleWeight):
  for i in range(epoch):
     if i % 5 == 0:
       intermediateImage = target.data
       intermediateImage = im_convert(intermediateImage,i)
       
     optimizer.zero_grad()
     target_features = get_feature(target, vgg_19,indexes)
     _ , ct,ht,wt = target_features[4].shape
     content_loss = torch.mean((target_features[4] - contentFeatures)**2)
     del target_features[4]
     style_loss = 0
 
     for j in range(len(style_gram_weights)):
 
       target_feature = target_features[j]
       target_gram_matrix = gram_matrix(target_feature)
       _, c , h , w = target_feature.shape
       
       layer_style_loss =style_gram_weights[j]*torch.mean((target_gram_matrix - style_gram[j])**2)/(c*h*w)
       style_loss += layer_style_loss
     totalLoss = contentWeight*content_loss + styleWeight*style_loss
     print(i , totalLoss.data)
     totalLoss.backward(retain_graph = True)
     optimizer.step()

In [18]:
train(2000,1e4,1e2)

a:0')
1485 tensor(966.5264, device='cuda:0')
1486 tensor(966.2419, device='cuda:0')
1487 tensor(967.2793, device='cuda:0')
1488tensor(967.0166, device='cuda:0')
1489 tensor(960.8626, device='cuda:0')
1490 tensor(975.0743, device='cuda:0')
1491 tensor(967.3707, device='cuda:0')
1492 tensor(981.4148, device='cuda:0')
1493tensor(983.0140, device='cuda:0')
1494 tensor(989.0687, device='cuda:0')
1495 tensor(998.0690, device='cuda:0')
1496 tensor(998.6495, device='cuda:0')
1497 tensor(989.1276, device='cuda:0')
1498tensor(996.1116, device='cuda:0')
1499 tensor(989.5811, device='cuda:0')
1500 tensor(998.2710, device='cuda:0')
1501 tensor(993.1476, device='cuda:0')
1502 tensor(993.2811, device='cuda:0')
1503tensor(991.2097, device='cuda:0')
1504 tensor(988.5375, device='cuda:0')
1505 tensor(994.3649, device='cuda:0')
1506 tensor(993.6931, device='cuda:0')
1507 tensor(998.0328, device='cuda:0')
1508tensor(992.5330, device='cuda:0')
1509 tensor(992.7362, device='cuda:0')
1510 tensor(990.7444, de